In [89]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import cross_val_score


## Read Data

In [90]:
df_train = pd.read_csv("../datasets/train.csv")
df_test = pd.read_csv("../datasets/test.csv")

# รวมข้อมูล
df = pd.concat([df_train, df_test], ignore_index=True)
df_copy = df.copy()
filtered_df = df_copy[(df_copy['Class'] == 'Business')]
filtered_df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied
7,7,96462,Female,Loyal Customer,52,Business travel,Business,2035,4,3,...,5,5,5,5,4,5,4,4,0.0,satisfied


## Understanding Data

In [66]:
filtered_df.keys()

Index(['Unnamed: 0', 'id', 'Gender', 'Customer Type', 'Age', 'Type of Travel',
       'Class', 'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction'],
      dtype='object')

In [67]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62160 entries, 1 to 129878
Data columns (total 25 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         62160 non-null  int64  
 1   id                                 62160 non-null  int64  
 2   Gender                             62160 non-null  object 
 3   Customer Type                      62160 non-null  object 
 4   Age                                62160 non-null  int64  
 5   Type of Travel                     62160 non-null  object 
 6   Class                              62160 non-null  object 
 7   Flight Distance                    62160 non-null  int64  
 8   Inflight wifi service              62160 non-null  int64  
 9   Departure/Arrival time convenient  62160 non-null  int64  
 10  Ease of Online booking             62160 non-null  int64  
 11  Gate location                      62160 non-null  int64  

In [68]:
filtered_df.describe().round(2)

,Unnamed: 0,id,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
count,62160.00,62160.00,62160.00,62160.00,62160.00,62160.00,62160.00,62160.00,62160.00,62160.00,62160.00,62160.00,62160.00,62160.00,62160.00,62160.00,62160.00,62160.00,62160.00,61990.00
mean,44143.39,69090.04,41.62,1676.09,2.78,2.91,2.92,2.99,3.33,3.72,3.76,3.64,3.68,3.65,3.84,3.52,3.85,3.48,14.30,14.49
std,31225.34,37842.46,12.75,1136.65,1.43,1.51,1.48,1.37,1.28,1.22,1.19,1.24,1.19,1.22,1.12,1.19,1.12,1.24,37.69,37.98
min,1.00,1.00,7.00,56.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
25%,16190.75,37982.75,32.00,600.00,2.00,2.00,2.00,2.00,2.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,0.00,0.00
50%,38864.50,72477.50,42.00,1589.00,3.00,3.00,3.00,3.00,3.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,0.00,0.00
75%,71414.75,101332.25,51.00,2558.00,4.00,4.00,4.00,4.00,4.00,5.00,5.00,5.00,5.00,5.00,5.00,4.00,5.00,4.00,12.00,12.00
max,103903.00,129871.00,85.00,4983.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1305.00,1280.00


In [69]:
filtered_df.isnull().sum()

Unnamed: 0                             0
id                                     0
Gender                                 0
Customer Type                          0
Age                                    0
Type of Travel                         0
Class                                  0
Flight Distance                        0
Inflight wifi service                  0
Departure/Arrival time convenient      0
Ease of Online booking                 0
Gate location                          0
Food and drink                         0
Online boarding                        0
Seat comfort                           0
Inflight entertainment                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Inflight service                       0
Cleanliness                            0
Departure Delay in Minutes             0
Arrival Delay in Minutes             170
satisfaction    

In [70]:
df_copy_cleaned = filtered_df.dropna()
df_copy_cleaned=df_copy_cleaned.drop(['Unnamed: 0','id'],axis=1).reset_index(drop=True)
df_copy_cleaned


,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
1,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
2,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
3,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied
4,Female,Loyal Customer,52,Business travel,Business,2035,4,3,4,4,...,5,5,5,5,4,5,4,4,0.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61985,Female,disloyal Customer,39,Business travel,Business,404,1,1,1,3,...,2,5,3,4,4,4,2,0,0.0,neutral or dissatisfied
61986,Male,Loyal Customer,52,Business travel,Business,280,3,3,3,3,...,4,4,4,4,3,4,3,0,0.0,satisfied
61987,Male,disloyal Customer,34,Business travel,Business,526,3,3,3,1,...,4,3,2,4,4,5,4,0,0.0,neutral or dissatisfied
61988,Male,Loyal Customer,23,Business travel,Business,646,4,4,4,4,...,4,4,5,5,5,5,4,0,0.0,satisfied


In [71]:
df_copy_cleaned.isnull().sum()

Gender                               0
Customer Type                        0
Age                                  0
Type of Travel                       0
Class                                0
Flight Distance                      0
Inflight wifi service                0
Departure/Arrival time convenient    0
Ease of Online booking               0
Gate location                        0
Food and drink                       0
Online boarding                      0
Seat comfort                         0
Inflight entertainment               0
On-board service                     0
Leg room service                     0
Baggage handling                     0
Checkin service                      0
Inflight service                     0
Cleanliness                          0
Departure Delay in Minutes           0
Arrival Delay in Minutes             0
satisfaction                         0
dtype: int64

In [72]:
df_copy_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61990 entries, 0 to 61989
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             61990 non-null  object 
 1   Customer Type                      61990 non-null  object 
 2   Age                                61990 non-null  int64  
 3   Type of Travel                     61990 non-null  object 
 4   Class                              61990 non-null  object 
 5   Flight Distance                    61990 non-null  int64  
 6   Inflight wifi service              61990 non-null  int64  
 7   Departure/Arrival time convenient  61990 non-null  int64  
 8   Ease of Online booking             61990 non-null  int64  
 9   Gate location                      61990 non-null  int64  
 10  Food and drink                     61990 non-null  int64  
 11  Online boarding                    61990 non-null  int

## OneHotEncoder

In [73]:
encoded_need=[]
for i in df_copy_cleaned.columns:
  if df_copy_cleaned[i].dtype=='object':
    print(f"Unique values of '{i}' is {df_copy_cleaned[i].unique()}")
    encoded_need.append(i)
print("Categorical values exist in the columns : ",encoded_need)
encoded_need = encoded_need[:-1]
print(encoded_need)

Unique values of 'Gender' is ['Male' 'Female']
Unique values of 'Customer Type' is ['disloyal Customer' 'Loyal Customer']
Unique values of 'Type of Travel' is ['Business travel' 'Personal Travel']
Unique values of 'Class' is ['Business']
Unique values of 'satisfaction' is ['neutral or dissatisfied' 'satisfied']
Categorical values exist in the columns :  ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'satisfaction']
['Gender', 'Customer Type', 'Type of Travel', 'Class']


In [88]:
max_values = df_copy_cleaned.max()

# หาคอลัมน์ที่มีค่า max = 5
columns_with_max_5 = max_values[max_values == 5].index.tolist()

print("คอลัมน์ที่มีค่า max เท่ากับ 5:", columns_with_max_5)
encoded_all = columns_with_max_5 + encoded_need
print(encoded_all)

คอลัมน์ที่มีค่า max เท่ากับ 5: ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']
['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness', 'Gender', 'Customer Type', 'Type of Travel', 'Class']


In [75]:
from sklearn.preprocessing import OneHotEncoder

encoder_ob = OneHotEncoder()

encoded_object = encoder_ob.fit_transform(df_copy_cleaned[encoded_need])

df_encoded_object = pd.DataFrame(encoded_object.toarray(), columns=encoder_ob.get_feature_names_out(encoded_need))
df_encoded_object = df_encoded_object.astype(int)
df_encoded_object

,Gender_Female,Gender_Male,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business
0,0,1,0,1,1,0,1
1,1,0,1,0,1,0,1
2,1,0,1,0,1,0,1
3,0,1,1,0,1,0,1
4,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...
61985,1,0,0,1,1,0,1
61986,0,1,1,0,1,0,1
61987,0,1,0,1,1,0,1
61988,0,1,1,0,1,0,1


In [93]:
encoder = OneHotEncoder()

encoded = encoder.fit_transform(df_copy_cleaned[encoded_all])

df_encoded = pd.DataFrame(encoded.toarray(), columns=encoder.get_feature_names_out(encoded_all))
df_encoded = df_encoded.astype(int)
print(list(df_encoded))

['Inflight wifi service_0', 'Inflight wifi service_1', 'Inflight wifi service_2', 'Inflight wifi service_3', 'Inflight wifi service_4', 'Inflight wifi service_5', 'Departure/Arrival time convenient_0', 'Departure/Arrival time convenient_1', 'Departure/Arrival time convenient_2', 'Departure/Arrival time convenient_3', 'Departure/Arrival time convenient_4', 'Departure/Arrival time convenient_5', 'Ease of Online booking_0', 'Ease of Online booking_1', 'Ease of Online booking_2', 'Ease of Online booking_3', 'Ease of Online booking_4', 'Ease of Online booking_5', 'Gate location_0', 'Gate location_1', 'Gate location_2', 'Gate location_3', 'Gate location_4', 'Gate location_5', 'Food and drink_0', 'Food and drink_1', 'Food and drink_2', 'Food and drink_3', 'Food and drink_4', 'Food and drink_5', 'Online boarding_0', 'Online boarding_1', 'Online boarding_2', 'Online boarding_3', 'Online boarding_4', 'Online boarding_5', 'Seat comfort_0', 'Seat comfort_1', 'Seat comfort_2', 'Seat comfort_3', 'Se

In [77]:
df_drop_object = df_copy_cleaned.drop(encoded_need, axis=1)
df_drop_object

,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,25,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
1,26,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,satisfied
2,25,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
3,61,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,satisfied
4,52,2035,4,3,4,4,5,5,5,5,5,5,5,4,5,4,4,0.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61985,39,404,1,1,1,3,2,1,2,2,5,3,4,4,4,2,0,0.0,neutral or dissatisfied
61986,52,280,3,3,3,3,3,4,4,4,4,4,4,3,4,3,0,0.0,satisfied
61987,34,526,3,3,3,1,4,3,4,4,3,2,4,4,5,4,0,0.0,neutral or dissatisfied
61988,23,646,4,4,4,4,4,4,4,4,4,5,5,5,5,4,0,0.0,satisfied


In [78]:
df_drop_object.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61990 entries, 0 to 61989
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                61990 non-null  int64  
 1   Flight Distance                    61990 non-null  int64  
 2   Inflight wifi service              61990 non-null  int64  
 3   Departure/Arrival time convenient  61990 non-null  int64  
 4   Ease of Online booking             61990 non-null  int64  
 5   Gate location                      61990 non-null  int64  
 6   Food and drink                     61990 non-null  int64  
 7   Online boarding                    61990 non-null  int64  
 8   Seat comfort                       61990 non-null  int64  
 9   Inflight entertainment             61990 non-null  int64  
 10  On-board service                   61990 non-null  int64  
 11  Leg room service                   61990 non-null  int

In [79]:
df_drop_dummy = df_copy_cleaned.drop(encoded_all, axis=1)
df_drop_dummy

,Age,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,25,235,1,6.0,neutral or dissatisfied
1,26,1142,0,0.0,satisfied
2,25,562,11,9.0,neutral or dissatisfied
3,61,214,0,0.0,satisfied
4,52,2035,4,0.0,satisfied
...,...,...,...,...,...
61985,39,404,0,0.0,neutral or dissatisfied
61986,52,280,0,0.0,satisfied
61987,34,526,0,0.0,neutral or dissatisfied
61988,23,646,0,0.0,satisfied


In [80]:
df_drop_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61990 entries, 0 to 61989
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         61990 non-null  int64  
 1   Flight Distance             61990 non-null  int64  
 2   Departure Delay in Minutes  61990 non-null  int64  
 3   Arrival Delay in Minutes    61990 non-null  float64
 4   satisfaction                61990 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 2.4+ MB


In [81]:
# ตรวจสอบขนาดของ df_drop_dummy และ df_encoded ก่อน
print("ขนาดของ df_drop_object:", df_drop_object.shape)
print("ขนาดของ df_encoded:", df_encoded_object.shape)

df_final_object = pd.concat([df_encoded_object, df_drop_object], axis=1)
print("ขนาดของ df_final:", df_final_object.shape)

ขนาดของ df_drop_object: (61990, 19)
ขนาดของ df_encoded: (61990, 7)
ขนาดของ df_final: (61990, 26)


In [82]:
# ตรวจสอบขนาดของ df_drop_dummy และ df_encoded ก่อน
print("ขนาดของ df_drop_dummy:", df_drop_dummy.shape)
print("ขนาดของ df_encoded:", df_encoded.shape)

df_final = pd.concat([df_encoded, df_drop_dummy], axis=1)
print("ขนาดของ df_final:", df_final.shape)


ขนาดของ df_drop_dummy: (61990, 5)
ขนาดของ df_encoded: (61990, 90)
ขนาดของ df_final: (61990, 95)


In [83]:
df_final_object

,Gender_Female,Gender_Male,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Age,Flight Distance,Inflight wifi service,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,1,0,1,1,0,1,25,235,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
1,1,0,1,0,1,0,1,26,1142,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
2,1,0,1,0,1,0,1,25,562,2,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
3,0,1,1,0,1,0,1,61,214,3,...,3,3,4,4,3,3,3,0,0.0,satisfied
4,1,0,1,0,1,0,1,52,2035,4,...,5,5,5,5,4,5,4,4,0.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61985,1,0,0,1,1,0,1,39,404,1,...,2,5,3,4,4,4,2,0,0.0,neutral or dissatisfied
61986,0,1,1,0,1,0,1,52,280,3,...,4,4,4,4,3,4,3,0,0.0,satisfied
61987,0,1,0,1,1,0,1,34,526,3,...,4,3,2,4,4,5,4,0,0.0,neutral or dissatisfied
61988,0,1,1,0,1,0,1,23,646,4,...,4,4,5,5,5,5,4,0,0.0,satisfied


In [84]:
df_final

,Inflight wifi service_0,Inflight wifi service_1,Inflight wifi service_2,Inflight wifi service_3,Inflight wifi service_4,Inflight wifi service_5,Departure/Arrival time convenient_0,Departure/Arrival time convenient_1,Departure/Arrival time convenient_2,Departure/Arrival time convenient_3,...,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Age,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,0,0,1,0,0,0,0,1,0,...,0,1,1,0,1,25,235,1,6.0,neutral or dissatisfied
1,0,0,1,0,0,0,0,0,1,0,...,1,0,1,0,1,26,1142,0,0.0,satisfied
2,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,1,25,562,11,9.0,neutral or dissatisfied
3,0,0,0,1,0,0,0,0,0,1,...,1,0,1,0,1,61,214,0,0.0,satisfied
4,0,0,0,0,1,0,0,0,0,1,...,1,0,1,0,1,52,2035,4,0.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61985,0,1,0,0,0,0,0,1,0,0,...,0,1,1,0,1,39,404,0,0.0,neutral or dissatisfied
61986,0,0,0,1,0,0,0,0,0,1,...,1,0,1,0,1,52,280,0,0.0,satisfied
61987,0,0,0,1,0,0,0,0,0,1,...,0,1,1,0,1,34,526,0,0.0,neutral or dissatisfied
61988,0,0,0,0,1,0,0,0,0,0,...,1,0,1,0,1,23,646,0,0.0,satisfied


## Save Data

In [85]:
df_final.to_csv('../datasets/data_dummy.csv', index=False)
df_final_object.to_csv('../datasets/data_dummy_object.csv', index=False)

## SAVE ONEHOT

In [86]:
# import pickle
# with open('../model/encoder.pkl', 'wb') as f:
#     pickle.dump(encoder, f)